## Beginning Steps

In [7]:
import sys
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import argparse
import psutil
import numpy as np

nb_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'data')
if nb_dir not in sys.path:
    sys.path.insert(0, nb_dir)
# print(sys.path)
import features_selection as fs
import make_dataset as md
import build_data as bd
import get_raw_data as grd
import data_classes

models_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'models')
if models_dir not in sys.path:
    sys.path.insert(0, models_dir)
import nn_real as nn

try:
    import horovod.tensorflow as hvd
except:
    print("Failed to import horovod module. "
          "%s is intended for use with Uber's Horovod distributed training "
          "framework. To create a Docker image with Horovod support see "
          "docker-examples/Dockerfile.horovod." % __file__)
    raise



In [8]:
RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'raw') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'processed')
RANDOM_SEED = 123  # Set the seed to get reproducable results.
DT_FLOAT = tf.float32
NP_FLOAT = np.dtype('float32')

print(RAW_DIR, PRO_DIR)

/home/ubuntu/MLMortgage/data/raw /home/ubuntu/MLMortgage/data/processed


In [2]:
import tensorflow as tf

FLAGS, UNPARSED = nn.update_parser(argparse.ArgumentParser())
print("UNPARSED", UNPARSED)
FLAGS.logdir=Path(str('/home/ubuntu/real_summaries4425-15ep_test/'))
if not os.path.exists(os.path.join(FLAGS.logdir)): #os.path.exists
    os.makedirs(os.path.join(FLAGS.logdir))
FLAGS = nn.FLAGS_setting(FLAGS, 1)
FLAGS.train_dir = 'chuncks_random_c1millx2_train'
FLAGS.valid_dir = 'chuncks_random_c1millx2_valid'
FLAGS.test_dir = 'chuncks_random_c1millx2_test'
FLAGS.train_period=[121,279] #[121, 143] 
FLAGS.valid_period=[280,285] #[144, 147] 
FLAGS.test_period=[286,304] #[148, 155]
FLAGS.epoch_num=15 
FLAGS.max_epoch_size=-1 
FLAGS.batch_size=4425
FLAGS.lr_decay_policy       = 'time'
FLAGS.lr_decay_epochs       = 30
FLAGS.lr_decay_rate         = 0.1
FLAGS.lr_poly_power         = 2.
print("FLAGS", FLAGS) #you can change the FLAGS by adding the setting before this line.

UNPARSED ['-f', '/run/user/1000/jupyter/kernel-d0fde1d7-74f4-4835-9dcf-945811b12ee7.json']
FLAGS Namespace(allow_summaries=True, batch_layer_type='batch', batch_norm=True, batch_size=4425, batch_type='layer', data_dir='/input_data', decay_rate=1, decay_step=800, dropout=True, dropout_keep=0.9, epoch_flag=0, epoch_num=15, learning_rate=0.1, log_file=<_io.TextIOWrapper name='/home/ubuntu/real_summaries4425-15ep_test/batch_layer_type_1_log.txt' mode='w+' encoding='UTF-8'>, logdir=PosixPath('/home/ubuntu/real_summaries4425-15ep_test'), loss_tolerance=0.0001, max_epoch_size=-1, momentum=0.5, n_hidden=3, name='batch_layer_type', rate_min=0.0015, reg_rate=1e-05, s_hidden=[200, 140, 140], stratified_flag=False, test_batch_size=100000, test_dir='chuncks_random_c1millx2_test', test_flag=True, test_period=[286, 304], train_dir='chuncks_random_c1millx2_train', train_period=[121, 279], valid_batch_size=100000, valid_dir='chuncks_random_c1millx2_valid', valid_period=[280, 285], weighted_sampling=Fal

In [9]:
class GPUNetworkBuilder(object):
    """This class provides convenient methods for constructing feed-forward
    networks with internal data layout of 'NCHW'.
    """
    def __init__(self,
                 # is_training,
                 dtype=tf.float32,
                 activation='RELU',
                 use_batch_norm=True,
                 batch_norm_config = {'decay':   0.9,
                                      'epsilon': 1e-4,
                                      'scale':   True,
                                      'zero_debias_moving_mean': False}):
        self.dtype             = dtype
        self.activation_func   = activation
        # self.is_training       = is_training
        self.use_batch_norm    = use_batch_norm
        self.batch_norm_config = batch_norm_config
        self._layer_counts     = defaultdict(lambda: 0)        
    
    def _count_layer(self, layer_type):
        idx  = self._layer_counts[layer_type]
        name = layer_type + str(idx)
        self._layer_counts[layer_type] += 1
        return name
    
    def variable_summaries(self, name, var, allow_summaries):
        """Create summaries for the given Tensor (for TensorBoard visualization (TB graphs)).
            Calculate the mean, min, max, histogram and standardeviation for 'var' variable and save the information
            in tf.summary.

        Args: 
             name (String): the of the scope for summaring. For min, max and standardeviation 'calculate_std' is used as sub-scope.
             var (Tensor): This is the tensor variable for building summaries.
        Returns: 
            None
        Raises:        
        """
        if allow_summaries:
            with tf.name_scope(name):
                mean = tf.reduce_mean(var)
                tf.summary.scalar('mean', mean)
                with tf.name_scope('calculate_std'):
                    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
                tf.summary.scalar('stddev', stddev)
                tf.summary.scalar('min', tf.reduce_min(var))
                tf.summary.scalar('max', tf.reduce_max(var))
                tf.summary.histogram('histogram', var)
                
    def _variable_on_cpu(self, name,
                     shape,
                     initializer=None,
                     regularizer=None,
                     dtype=DT_FLOAT):
        """Create a Variable or get an existing one stored on CPU memory.    

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
            initializer: Default None. Initializer for Variable.
            regularizer (A (Tensor -> Tensor or None) function): Default None. Regularizer for Variable.
            dtype (TYPE): Type of the new variable.
        Returns:
            Variable Tensor
        """
        with tf.device('/gpu:1'): # this operation is assigned to this device, but this make a copy of data when is transferred on and off the device, which is expensive.
            var = tf.get_variable(
                name,
                shape,
                initializer=initializer,
                regularizer=regularizer,
                dtype=dtype)
        return var

    def _create_variable(self, name,
                         shape, allow_summaries, 
                         initializer=None,
                         regularizer=None,
                         dtype=DT_FLOAT):
        """Call _variable_on_cpu methods and variable_summaries for the 'name' tensor variable. 

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
            initializer: Default None. Initializer for Variable.
            regularizer (A (Tensor -> Tensor or None) function): Default None. Regularizer for Variable.
            dtype (TYPE): Type of the new variable.
        Returns:
            Variable Tensor
        """
        var = _variable_on_cpu(name, shape, initializer, regularizer, dtype)
        variable_summaries(name + '/summaries', var, allow_summaries)
        return var

    def create_weights(self, name, shape, reg_rate, allow_summaries):
        """Create a Variable initialized with weights which are truncated normal distribution and regularized by
        l1_regularizer (L1 regularization encourages sparsity, Regularization can help prevent overfitting).    

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
        Returns:
            Variable Tensor
        """    
        dtype = DT_FLOAT
        # kernel_initializer = tf.uniform_unit_scaling_initializer(
        #     factor=1.43, dtype=DT_FLOAT)
        # kernel_initializer = tf.contrib.layers.xavier_initializer(
        #     uniform=True, dtype=DT_FLOAT)
        kernel_initializer = tf.truncated_normal_initializer(
            stddev=(1.0 / np.sqrt(shape[0])), dtype=dtype)

        regularizer = tf.contrib.layers.l2_regularizer(
            np.float32(reg_rate), 'penalty')
        return _create_variable(name, shape, allow_summaries, kernel_initializer, regularizer,
                                dtype)

    def bias_variable(self, name, shape, layer_name, weighted_sampling): # FLAGS.weighted_sampling
        """Create a bias variable with appropriate initialization. In case of FLAGS.weighted_sampling==False
        and layer_name contains 'soft' the bias variable will contain a np.array of Negative values. Otherwise
        the bias variable will be initialized in zero.

        Args:
            name (String): name of the variable.
            shape (list of ints): Shape of the variable.
            layer_name (String): name of the layer.
        Returns:
            Variable Tensor.
        """
        def initial_bias(layer_name):
            """Get the initial value for the bias of the layer with layer_name."""
            if (not weighted_sampling) and 'soft' in layer_name:
                return np.array(
                    [-4.66, -3.81, -4.81, -3.90, -0.08, -3.90, -7.51],
                    dtype=NP_FLOAT) + NP_FLOAT(4.1)
            return 0.0

        initial_value = initial_bias(layer_name)
        with tf.name_scope(name) as scope:
            initial = tf.constant(initial_value, shape=shape)
            bias = tf.Variable(initial, name=scope)
            variable_summaries('summaries', bias)
        return bias        
    
    def dropout_layer(self, name, tensor_before, FLAGS):
        """Compute dropout to tensor_before with name scoping and a placeholder for keep_prob. 
        With probability keep_prob, outputs the input element scaled up by 1 / keep_prob, otherwise outputs 0.

        Args:
            name (String): name of the scope.
            tensor_before (Tensor): Variable Tensor.        
        Returns:
            Variable Tensor of the same shape of tensor_before.
        """   
        if not FLAGS.dropout:
            print('There is not dropout for' + name)
            return tensor_before
        with tf.name_scope(name) as scope:
            keep_prob = tf.placeholder(DT_FLOAT, None, name='keep_proba')
            tf.summary.scalar('keep_probability', keep_prob)
            dropped = tf.nn.dropout(tensor_before, keep_prob=keep_prob, name=scope)
            variable_summaries('input_dropped_out', dropped, FLAGS.allow_summaries)
        return dropped

    def batch_normalization(self, name, input_tensor, train_flag, FLAGS):
        """Perform batch normalization over the input tensor.
        Batch normalization helps avoid overfitting and we're able to use more
        aggressive (larger) learning rates, resulting in faster convergence.
        training parameter: Either a Python boolean, or a TensorFlow boolean scalar tensor (e.g. a placeholder). 
        Whether to return the output in training mode (normalized with statistics of the current batch) or in 
        inference mode (normalized with moving statistics). NOTE: make sure to set this parameter correctly, 
        or else your training/inference will not work properly.

        Args:
            name (String): name of the scope and the name of the layer.
            input_tensor (Tensor): Variable Tensor.        
        Returns:
            Variable Tensor # the same shape of input_tensor??.
        """
        # if not FLAGS.batch_norm:
        #     return input_tensor
        # train_flag = tf.get_default_graph().get_tensor_by_name('train_flag:0')
        with tf.name_scope(name):
            normalized = tf.layers.batch_normalization(
                input_tensor,
                center=True,
                scale=True,
                training=train_flag,
                name=name)  # renorm=True, renorm_momentum=0.99)
            variable_summaries('normalized_batch', normalized, FLAGS.allow_summaries)
        return normalized

    def layer_normalization(self, name, input_tensor, FLAGS):
        """Perform layer normalization.

        Layer normalization helps avoid overfitting and we're able to use more
        aggressive (larger) learning rates, resulting in faster convergence.
        Can be used as a normalizer function for conv2d and fully_connected.

        Given a tensor inputs of rank R, moments are calculated and normalization 
        is performed over axes begin_norm_axis ... R - 1. 
        Scaling and centering, if requested, is performed over axes begin_params_axis .. R - 1.
        """
        # if not FLAGS.batch_norm:
        #     return input_tensor
        with tf.name_scope(name):
            normalized = tf.contrib.layers.layer_norm(
                input_tensor, center=True, scale=True, scope=name)
            variable_summaries('normalized_layer', normalized, FLAGS.allow_summaries)
        return normalized


    def normalize(self, name, input_tensor, train_flag, FLAGS):
        """Perform either type (batch/layer) of normalization."""
        if not FLAGS.batch_norm:
            return input_tensor
        if FLAGS.batch_type.lower() == 'batch':
            return batch_normalization(name, input_tensor, train_flag, FLAGS)
        if FLAGS.batch_type.lower() == 'layer':
            return layer_normalization(name, input_tensor, FLAGS)
        raise ValueError('Invalid value for batch_type: ' + FLAGS.batch_type)

    def nn_layer(self, input_tensor, output_dim, layer_name, FLAGS, act, train_flag):
        """Create a simple neural net layer.

        It performs the affine transformation and uses the activation function to
        nonlinearize. It further sets up name scoping so that the resultant graph
        is easy to read, and adds a number of summary ops.
        """
        input_dim = input_tensor.shape[1].value    
        with tf.variable_scope(layer_name): # A context manager for defining ops that creates variables (layers).
            weights = self.create_weights('weights', [input_dim, output_dim], FLAGS.reg_rate, FLAGS.allow_summaries)
            # This is outdated and no longer applies: Do not change the order of
            # batch normalization and drop out. batch # normalization has to stay
            # __before__ the drop out layer.
            self.variable_summaries('input', input_tensor, FLAGS.allow_summaries)
            input_tensor = self.dropout_layer('dropout', input_tensor, FLAGS)
            with tf.name_scope('mix'):
                mixed = tf.matmul(input_tensor, weights)
                tf.summary.histogram('maybe_guassian', mixed)
            # Batch or layer normalization has to stay __after__ the affine
            # transformation (the bias term doens't really matter because of the
            # beta term in the normalization equation).
            # See pp. 5 of the batch normalization paper:
            # ```We add the BN transform immediately before the nonlinearity, by
            # normalizing x = W u + b```
            # biases = bias_variable('biases', [output_dim], layer_name)
            preactivate = self.normalize('layer_normalization', mixed, train_flag, FLAGS)  # + biases
            # tf.summary.histogram('pre_activations', preactivate)
            # preactivate = dropout_layer('dropout', preactivate)
            with tf.name_scope('activation') as scope:
                activations = self.activate(preactivate, funcname=act)
                tf.summary.histogram('activations', activations)
        return activations        
    
    def activate(self, input_layer, funcname=None):
        """Applies an activation function"""
        if isinstance(funcname, tuple):
            funcname = funcname[0]
            params = funcname[1:]
        if funcname is None:
            funcname = self.activation_func
        if funcname == 'LINEAR':
            return input_layer
        activation_map = {
            'IDENT':   tf.identity,
            'RELU':    tf.nn.relu,
            'RELU6':   tf.nn.relu6,
            'ELU':     tf.nn.elu,
            'SIGMOID': tf.nn.sigmoid,
            'TANH':    tf.nn.tanh,
            'LRELU':   lambda x, name: tf.maximum(params[0]*x, x, name=name)
        }
        return activation_map[funcname](input_layer, name=funcname.lower())
    
    def add_hidden_layers(features, architecture, FLAGS, train_flag, act=None):
        """Add hidden layers to the model using the architecture parameters."""
        hidden_out = features
        jit_scope = tf.contrib.compiler.jit.experimental_jit_scope #JIT compiler compiles and runs parts of TF graphs via XLA, fusing multiple operators (kernel fusion) nto a small number of compiled kernels.
        with jit_scope(): #this operation will be compiled with XLA.
            for hid_i in range(1, FLAGS.n_hidden + 1):
                hidden_out = self.nn_layer(hidden_out,
                                      architecture['n_hidden_{:1d}'.format(hid_i)],
                                      '{:1d}_hidden'.format(hid_i), FLAGS, act, train_flag)
        return hidden_out        
    

In [ ]:
class FeedForwardTrainer(object):
    def __init__(self, loss_func, global_step=None, nstep_per_epoch=None):        
        self.loss_func = loss_func
#        with tf.device('/cpu:0'):
#            self.global_step = tf.get_variable(
#                'global_step', [],
#                initializer=tf.constant_initializer(0),
#                dtype=tf.int64,
#                trainable=False)

    def get_learning_rate(initial_learning_rate, epoch_flag):
        """Get the learning rate."""
        with tf.name_scope('learning_rate') as scope:
            if FLAGS.lr_decay_policy == 'poly':
                self.learning_rate = tf.train.polynomial_decay(
                                        initial_learning_rate,
                                        epoch_flag,
                                        decay_steps=FLAGS.epoch_num*nstep_per_epoch,
                                        end_learning_rate=0.,
                                        power=FLAGS.lr_poly_power,
                                        cycle=False)
            elif FLAGS.lr_decay_policy == 'exp':
                self.learning_rate = tf.train.exponential_decay(
                                        initial_learning_rate,
                                        epoch_flag,
                                        decay_steps=FLAGS.lr_decay_epochs*nstep_per_epoch,
                                        decay_rate=FLAGS.lr_decay_rate,
                                        staircase=True)
            else:            
                # decayed_lr = tf.train.exponential_decay(
                #     initial_learning_rate,
                #     global_step,
                #     FLAGS.decay_step,
                #     FLAGS.decay_base,
                #     staircase=False)
                decayed_lr = tf.train.inverse_time_decay(
                    initial_learning_rate,
                    epoch_flag,
                    decay_steps=FLAGS.decay_step,
                    decay_rate=FLAGS.decay_rate)
                final_lr = tf.clip_by_value(
                    decayed_lr, FLAGS.rate_min, 1000, name=scope)
                tf.summary.scalar('value', final_lr)
                self.learning_rate = final_lr
        # return self.learning_rate 

    def get_accuracy(self, labels_int, logits, name):
        """Get the accuracy tensor."""
        with tf.name_scope(name) as scope:
            # For a classifier model, we can use the in_top_k Op.
            # It returns a bool tensor with shape [batch_size] that is true for
            # the examples where the label is in the top k (here k=1)
            # of all logits for that example.
            correct = tf.nn.in_top_k(
                logits, labels_int, 1, name='correct_prediction') # returns a tensor of type bool.
            return tf.reduce_mean(tf.cast(correct, DT_FLOAT), name=scope)

    # auc = get_auc(labels, probs, True, 'metrics/auc')
    def get_auc(self, labels, scores, hist_flag, name):
        """Calculate the AUC of the two-way classifier for the given class."""

        def get_auc_using_histogram(labels, scores, class_, name):
            """Calculate the AUC."""
            class_ind = class_dict[class_.upper()]
            with tf.name_scope(name) as scope:
                auc, update_op = tf.contrib.metrics.auc_using_histogram( # his Op maintains Variables containing histograms of the scores associated with True and False labels. 
                    tf.cast(labels[:, class_ind], tf.bool),
                    scores[:, class_ind],
                    score_range=[0.0, 1.0],
                    nbins=200,
                    collections=None,
                    name=scope)
            ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
            # print(update_op.name)
            # print(auc) # it doesn't work because FailedPreconditionError (see above for traceback): Attempting to use uninitialized value metrics/auc/0//hist_accumulate/hist_true_acc
            # aucp = tf.Print(auc,[auc], message='AUC the label: ' + class_) # it doesnt work because it doesnt run in a session
            # print(aucp)
            return auc

        def get_auc_metric(labels, scores, class_, name):
            """Determine the AUC using conventional methods."""
            class_ind = class_dict[class_.upper()]
            with tf.name_scope(name) as scope:
                auc, _ = tf.metrics.auc( # Computes the approximate AUC via a Riemann sum.
                    tf.cast(labels[:, class_ind], tf.bool), # ?? Print out!!
                    scores[:, class_ind],
                    weights=None,
                    num_thresholds=200,
                    metrics_collections=None,
                    updates_collections=ops.GraphKeys.UPDATE_OPS,
                    curve='ROC',
                    name=scope)
            # print(auc.op.name)
            return auc

        classes = ['0', '3', '6', '9', 'C', 'F', 'R']
        class_dict = {classes[ind]: ind for ind in range(len(classes))}
        if hist_flag:
            auc_func = get_auc_using_histogram
        else:
            auc_func = get_auc_metric
        with tf.name_scope(name) as scope:
            aucv = [
                    auc_func(labels, scores, class_, str(ind)) for ind, class_ in enumerate(classes) # pair (index ej. 0, value ej. '0')
                   ]      
            auc_values = tf.stack( # Pack along first dim
                aucv,
                axis=0,
                name=scope)
            # aucv = tf.Print(auc_values,[auc_values], message='AUC for all labels: ')
            # print(aucv) # or maybe aucv.eval() or var = tf.Variable(aucv) and then var.eval(session=sess), or ovar = sess.run(var) but Attempting to use uninitialized value metrics/auc/Variable
            return auc_values


    # conf_mtx = get_confusion_matrix(labels_int, predictions, len(classes), 'metrics/confusion')
    def get_confusion_matrix(self, labels_int, predictions, num_classes, name):
        """Get the confusion matrix.
        Both prediction and labels must be 1-D arrays of the same shape in order for 
        this function to work.
        """
        with tf.name_scope(name) as scope:
            conf = tf.confusion_matrix(
                labels_int,
                predictions=predictions,
                num_classes=num_classes,
                dtype=tf.int32,
                name=scope,
                weights=None)
        # print(conf.op.name)
        return conf #return a K x K Matriz K = num_classes


    def get_m_hand(self, labels, scores, name):
        """Implement the M measure described in Hand.

        See ```A Simple Generalisation of the Area Under the ROC Curve for Multiple
        Class Classification Problems``` Hand, Till 2001.    

        """
        def get_auc_using_histogram(labels, scores, first_ind, second_ind, scope):
            """Calculate the AUC.
            Calculate the AUC value by maintainig histograms of boolean variables (labels and 
            scores masked by the First-Second Individuals rule).
            """
            mask = (labels[:, first_ind] + labels[:, second_ind]) > 0 #one in at least one column.
            auc, update_op = tf.contrib.metrics.auc_using_histogram( # maintains variables containing histograms of the scores associated with True, False labels. 
                tf.cast(tf.boolean_mask(labels[:, first_ind], mask), tf.bool), # tf.boolean_mask: Apply boolean mask to tensor. Numpy equivalent is tensor[mask].
                tf.boolean_mask(scores[:, first_ind], mask),
                score_range=[0.0, 1.0],
                nbins=500,
                collections=None,
                name=scope)
            ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
            # print(update_op.name)
            return auc

        temp_array = []
        with tf.name_scope(name) as main_scope:
            for first_ind in range(7):
                for second_ind in range(7):
                    if first_ind != second_ind:
                        final_name = '{:d}{:d}'.format(first_ind, second_ind)
                        with tf.name_scope(final_name) as scope:
                            auc = get_auc_using_histogram(
                                labels, scores, first_ind, second_ind, scope)
                        temp_array.append(auc)
            return tf.stack(temp_array, axis=0, name=main_scope) # Stacks a list of rank-R tensors into one rank-(R+1) tensor.


    def get_auc_pr_curve(self, labels, scores, name, num_thresholds):    
        with tf.name_scope(name) as scope:                             
            AUC_PR = []
            AUC_data = []
            for i in range(7):  
                data, update_op = tf.contrib.metrics.precision_recall_at_equal_thresholds(
                                name='pr_data',
                                predictions=scores[:, i],
                                labels=tf.cast(labels[:, i], tf.bool),
                                num_thresholds=10, use_locking=True)
                ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
                AUC_data.append((tf.stack(data.recall), tf.stack(data.precision), tf.stack(data.thresholds)))   # we cant use sklearn with tensorflow definition!
                auc, _ = tf.metrics.auc(labels[:, i], scores[:, i], weights=None, num_thresholds=10, 
                                        curve='PR', updates_collections=ops.GraphKeys.UPDATE_OPS, metrics_collections=None, summation_method='careful_interpolation') # 
                # ops.add_to_collections(ops.GraphKeys.UPDATE_OPS, update_op)
                AUC_PR.append(auc)
            # print(AUC_data)
            return tf.stack( # Pack the array of scalar tensor along one dim tensor
                AUC_PR,
                axis=0,
                name=scope), AUC_data

    def log_loss(self, labels, probs, name):
        """
        Args:
            labels: Labels tensor, int32 - [batch_size, n_classes], with one-hot
            encoded values.
            logits: Probabilities tensor, float32 - [batch_size, n_classes].
        """
        with tf.name_scope(name) as scope:
            total_loss = 0
            for j in range(probs.shape[1].value):
                loss = tf.losses.log_loss(labels[:, j], probs[:, j], loss_collection=None)
                total_loss += loss

            return tf.div(total_loss, np.float32(probs.shape[1].value), name=scope)
    
    def calculate_metrics(self, labels, logits):
        """Evaluate the quality of the logits at predicting the label.

        Args:
            labels: Labels tensor, int32 - [batch_size, n_classes], with one-hot
            encoded values.
            logits: Logits tensor, float32 - [batch_size, n_classes].
        Returns:
            A scalar float32 tensor with the fraction of examples (out of
            batch_size) that were predicted correctly.
        """
        classes = ['0', '3', '6', '9', 'C', 'F', 'R']
        with tf.name_scope('metrics'):
            labels_int = tf.argmax(labels, 1, name='intlabels') #tf.argmax: Returns the index with the largest value across axes=1 of a tensor.		
            predictions = tf.argmax(logits, 1, name='predictions')        
            probs = tf.nn.softmax(logits, name='probs') # Computes softmax activations. softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), axis)        

        m_list = self.get_m_hand(labels, probs, 'metrics/m_measure')
        accuracy = self.get_accuracy(labels_int, logits, 'metrics/accuracy')    
        auc = self.get_auc(labels, probs, True, 'metrics/auc')    
        conf_mtx = self.get_confusion_matrix(labels_int, predictions,
                                        len(classes), 'metrics/confusion')
        loss = self.log_loss(labels, probs, 'metrics/log_loss')
        pr_auc, pr_data = self.get_auc_pr_curve(labels, probs, 'metrics/auc_pr', 200)

        # this is for the definition of the graph:
        return accuracy, conf_mtx, auc, m_list, loss, pr_auc, pr_data
    
    def training_step(self, batch_size,  architecture, FLAGS, epoch_flag):        
        features = tf.placeholder(
            DT_FLOAT, [None, architecture['n_input']], name='features')
        labels = tf.placeholder(
            DT_FLOAT, [None, architecture['n_classes']], name='targets')
        epoch_flag = tf.placeholder(tf.int32, None, name='epoch_flag')
        example_weights = tf.placeholder(
            DT_FLOAT, [None], name='example_weights')
        with tf.device('/gpu:0'):
            # Evaluate the loss and compute the gradients            
            loss, logits = self.loss_func(features, labels,  architecture, FLAGS)

        with tf.device('/cpu:0'): # No in_top_k implem on GPU
            accuracy, conf_mtx, auc_list, m_list, lloss, auc_pr, auc_data = self.calculate_metrics(labels, logits)
            with tf.name_scope('0_performance'):
                # Scalar summaries to track the loss and accuracy over time in TB.
                tf.summary.scalar('0accuracy', accuracy)
                tf.summary.scalar('1better_accuracy',
                                  tf.reduce_mean(
                                      tf.diag_part(conf_mtx / tf.reduce_sum(
                                          conf_mtx, axis=1, keepdims=True))))
                tf.summary.scalar('2auc_aoc', tf.reduce_mean(auc_list))
                tf.summary.scalar('3m_measure', tf.reduce_mean(m_list))
                tf.summary.scalar('4loss', loss)
                tf.summary.scalar('5log_loss', lloss)
                tf.summary.scalar('6auc_pr', tf.reduce_mean(auc_pr))

        # Apply the gradients to optimize the loss function
        with tf.device('/gpu:0'):            
            update_ops = ops.get_collection(ops.GraphKeys.UPDATE_OPS)
            # print(update_ops)
            with ops.control_dependencies(update_ops):
                with tf.name_scope('train') as scope:
                    # print_loss = tf.Print(loss, [loss], name='print_loss') 

                    # Create a variable to track the global step.
        #            global_step = tf.get_variable(
        #                'train/global_step',
        #                shape=[],
        #                initializer=tf.constant_initializer(0, dtype=tf.int32),
        #                trainable=False)            

                    final_learning_rate = self.learning_rate # self.get_learning_rate(FLAGS.learning_rate, epoch_flag)

                    # optimizer = tf.train.GradientDescentOptimizer(final_learning_rate)
                    optimizer = tf.train.MomentumOptimizer(final_learning_rate, FLAGS.momentum, use_nesterov=True)
                    # optimizer = tf.train.AdagradOptimizer(final_learning_rate)

                    # Use the optimizer to apply the gradients that minimize the loss
                    # (and increment the global step counter) as a single training step.
        #            return optimizer.minimize(
        #                loss, global_step=global_step, name=scope)
                    optimizer = hvd.DistributedOptimizer(optimizer) #HVD!!
                    train_op = optimizer.minimize(loss, global_step=None, name=scope)
            
                        
        return loss, self.learning_rate, train_op
    
    def init(self, sess):
        # init_op = tf.global_variables_initializer()
        # sess.run(init_op)        
        """Add an Op to the graph to initialize the global and local variables."""
        with tf.name_scope('init') as scope:
            with tf.name_scope('global'):
                global_init = tf.global_variables_initializer()
            with tf.name_scope('local'):
                local_init = tf.local_variables_initializer()
                # print(local_init.name)
            tf.group(global_init, local_init, name=scope)
        
    def sync(self, sess):
        sync_op = hvd.broadcast_global_variables(0)
        sess.run(sync_op)

In [10]:
def loss_func(features, labels, architecture, FLAGS):
    # Build the forward model
    net = GPUNetworkBuilder(dtype=DT_FLOAT)
    train_flag = tf.placeholder(tf.bool, None, name='train_flag')
    with tf.name_scope('input_normalization') as scope:
        feature_norm = features
        variable_summaries('input_normalized', feature_norm, FLAGS.allow_summaries)
    hidden_out = net.add_hidden_layers(feature_norm, architecture, FLAGS, train_flag)
    # Linear output layer for the logits
    logits = (net.nn_layer(hidden_out, architecture['n_classes'],'9_softmax_linear', FLAGS, 'IDENT', train_flag))
    
    with tf.name_scope('loss') as scope:
        with tf.name_scope('regularization'):
            penalty = tf.losses.get_regularization_loss(name='penalty') #Gets the total regularization loss from an optional scope name (sum for ol + 3h + 2h + 1h).
            tf.summary.scalar('weight_norm', penalty / (1e-8 + reg_rate)) #for printing out
        with tf.name_scope('cross_entropy') as xentropy_scope:
            weighted_cross_entropy = tf.losses.softmax_cross_entropy(
                onehot_labels=labels,
                logits=logits,
                weights=1.0,  # weights,  #
                scope=xentropy_scope,
                loss_collection=ops.GraphKeys.LOSSES)
            tf.summary.scalar('weighted_cross_entropy', weighted_cross_entropy)
        loss= tf.add(weighted_cross_entropy, penalty, name=scope) # Returns x + y element-wise.    
            
    return loss, logits

## Training
From console, execute:

#### $ cd /home/ubuntu/MLMortgage/src/models

#### $ python nn_real.py --train_dir=chuncks_random_c1mill --valid_dir=chuncks_random_c1mill --test_dir=chuncks_random_c1mill --logdir=/home/ubuntu/real_summaries_4425_-15ep_99-01/ --epoch_num=15 --max_epoch_size=-1 --batch_size=4425                                                    
This execution runs 15 epochs over the entire dataset (max_epoch_size=-1) and the training, validation and testing datasets are in the same directory inside /home/ubuntu/MLMortgage/data/processed/chuncks_random_c1mill/. 

The checkpoints and the models results will be saved into, for example, logdir=/home/ubuntu/real_summaries_4425_-15ep_99-01/. You can change it by uncommenting and modifying the FLAGS.logdir variable.

To execute step by step:


In [4]:
DATA = md.get_h5_data(PRO_DIR, FLAGS.train_dir, FLAGS.valid_dir, FLAGS.test_dir, train_period=FLAGS.train_period, valid_period=FLAGS.valid_period, test_period=FLAGS.test_period) 
print('Features List: ', DATA.train.features_list)
print('Labels List: ', DATA.train.labels_list)

dict:  {'path': '/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/c1mill99-01_train_non_index_0.h5', 'nrows': 20809545, 'init_index': 0, 'end_index': 20809545, 'dataset': <class 'pandas.io.pytables.HDFStore'>
File path: /home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/c1mill99-01_train_non_index_0.h5
, 'dataset_features': array([[0.11811024, 0.        , 1.431     , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.3775    , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.354     , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 2.125     , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.125     , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.125     , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), 'dataset_labels': array([[0, 0, 0, ..., 1, 

In [5]:
FLAGS.log_file.write('METRICS:  %s\r\n' % str(FLAGS))
FLAGS.log_file.write('training files:  %s\r\n' % str(DATA.train._dict))
# print('training files:  %s\r\n' % str(DATA.train._dict))
FLAGS.log_file.write('validation files:  %s\r\n' % str(DATA.validation._dict))
# print('validation files:  %s\r\n' % str(DATA.validation._dict))
FLAGS.log_file.write('testing files:  %s\r\n' % str(DATA.test._dict))        
# print('testing files:  %s\r\n' % str(DATA.test._dict))     

1016

In [6]:
print('Training features - Sample', DATA.train._dict[0]['dataset_features'][0:100]) #you can increase the sampling number of records 

Training features - Sample [[0.11811024 0.         1.431      ... 0.         0.         0.        ]
 [0.         0.         1.3775     ... 0.         0.         0.        ]
 [0.         0.         1.354      ... 0.         0.         0.        ]
 ...
 [0.         0.         1.345      ... 0.         0.         0.        ]
 [0.         0.         1.345      ... 0.         0.         0.        ]
 [0.         0.         1.345      ... 0.         0.         0.        ]]


In [7]:
architecture = nn.architecture_settings(DATA, FLAGS)
print('RAM before build: ', psutil.virtual_memory()) #  physical memory usage
FLAGS.log_file.write('RAM  before build: %s\r\n' % str(psutil.virtual_memory()))
graph = nn.build_graph(architecture, FLAGS)        
print('RAM after build', psutil.virtual_memory()) #  physical memory usage
FLAGS.log_file.write('RAM  after build: %s\r\n' % str(psutil.virtual_memory()))
nn.run_model(graph, 'testing_data', 1,  FLAGS, DATA)      

architecture {'n_input': 258, 'n_classes': 7, 'n_hidden_1': 200, 'n_hidden_2': 140, 'n_hidden_3': 140}
RAM before build:  svmem(total=64389107712, available=24191283200, percent=62.4, used=39632322560, free=20344016896, active=42012483584, inactive=1591767040, buffers=87318528, cached=4325449728, shared=81408000, slab=98668544)
Building the computation graph in TF....


2018-10-05 20:15:50,063 - tensorflow - WARNING - From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/contrib/metrics/python/ops/histogram_ops.py:83: remove_squeezable_dimensions (from tensorflow.contrib.framework.python.framework.tensor_util) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to remove_squeezable_dimensions from tf.confusion_matrix. Note that the order of the inputs and outputs of labels and predictions have also been switched.


Graph building completed....
RAM after build svmem(total=64389107712, available=24044855296, percent=62.7, used=39778754560, free=20197855232, active=42158907392, inactive=1591345152, buffers=87343104, cached=4325154816, shared=81408000, slab=98693120)
before init tf variables:  svmem(total=64389107712, available=23930417152, percent=62.8, used=39891005440, free=20075352064, active=42188754944, inactive=1601404928, buffers=87343104, cached=4335407104, shared=83505152, slab=104845312)
after init tf variables:  svmem(total=64389107712, available=23921340416, percent=62.8, used=39898963968, free=20066275328, active=42197340160, inactive=1602404352, buffers=87343104, cached=4336525312, shared=84623360, slab=104849408)
Initialized all the local and global variables in the graph....
Epoch:  0
batch Number:  0
Time for Getting  4425  random elements:  0:00:00.007087
Batch Time:  0:00:10.708006
batch Number:  1
Time for Getting  4425  random elements:  0:00:00.007964
Batch Time:  0:00:00.14681

Batch Time:  0:00:00.145821
batch Number:  73
Time for Getting  4425  random elements:  0:00:00.007641
Batch Time:  0:00:00.145995
batch Number:  74
Time for Getting  4425  random elements:  0:00:00.007779
Batch Time:  0:00:00.147051
batch Number:  75
Time for Getting  4425  random elements:  0:00:00.008058
Batch Time:  0:00:00.213304
batch Number:  76
Time for Getting  4425  random elements:  0:00:00.007806
Batch Time:  0:00:00.150429
batch Number:  77
Time for Getting  4425  random elements:  0:00:00.007771
Batch Time:  0:00:00.148448
batch Number:  78
Time for Getting  4425  random elements:  0:00:00.007718
Batch Time:  0:00:00.142374
batch Number:  79
Time for Getting  4425  random elements:  0:00:00.007688
Batch Time:  0:00:00.143903
batch Number:  80
Time for Getting  4425  random elements:  0:00:00.007556
Batch Time:  0:00:00.145970
batch Number:  81
Time for Getting  4425  random elements:  0:00:00.007771
Batch Time:  0:00:00.143949
batch Number:  82
Time for Getting  4425  ran

Batch Time:  0:00:00.142571
batch Number:  154
Time for Getting  4425  random elements:  0:00:00.007583
Batch Time:  0:00:00.149103
batch Number:  155
Time for Getting  4425  random elements:  0:00:00.007743
Batch Time:  0:00:00.148587
batch Number:  156
Time for Getting  4425  random elements:  0:00:00.007637
Batch Time:  0:00:00.150524
batch Number:  157
Time for Getting  4425  random elements:  0:00:00.007721
Batch Time:  0:00:00.143935
batch Number:  158
Time for Getting  4425  random elements:  0:00:00.007824
Batch Time:  0:00:00.142715
batch Number:  159
Time for Getting  4425  random elements:  0:00:00.007706
Batch Time:  0:00:00.151532
batch Number:  160
Time for Getting  4425  random elements:  0:00:00.008030
Batch Time:  0:00:00.146588
batch Number:  161
Time for Getting  4425  random elements:  0:00:00.008084
Batch Time:  0:00:00.149098
batch Number:  162
Time for Getting  4425  random elements:  0:00:00.007798
Batch Time:  0:00:00.147442
batch Number:  163
Time for Getting 

...

Batch Time:  0:00:00.140863
batch Number:  1665
Time for Getting  4425  random elements:  0:00:00.007529
Batch Time:  0:00:00.138312
batch Number:  1666
Time for Getting  4425  random elements:  0:00:00.007434
Batch Time:  0:00:00.139778
batch Number:  1667
Time for Getting  4425  random elements:  0:00:00.007517
Batch Time:  0:00:00.143346
batch Number:  1668
Time for Getting  4425  random elements:  0:00:00.007453
Batch Time:  0:00:00.143850
batch Number:  1669
Time for Getting  4425  random elements:  0:00:00.007397
Batch Time:  0:00:00.135800
batch Number:  1670
Time for Getting  4425  random elements:  0:00:00.007189
Batch Time:  0:00:00.142431
batch Number:  1671
Time for Getting  4425  random elements:  0:00:00.007489
Batch Time:  0:00:00.143058
batch Number:  1672
Time for Getting  4425  random elements:  0:00:00.007565
Batch Time:  0:00:00.139037
batch Number:  1673
Time for Getting  4425  random elements:  0:00:00.007494
Batch Time:  0:00:00.142405
batch Number:  1674
Time fo

Batch Time:  0:00:00.145257
batch Number:  1745
Time for Getting  4425  random elements:  0:00:00.007843
Batch Time:  0:00:00.141384
batch Number:  1746
Time for Getting  4425  random elements:  0:00:00.007600
Batch Time:  0:00:00.146829
batch Number:  1747
Time for Getting  4425  random elements:  0:00:00.007505
Batch Time:  0:00:00.139990
batch Number:  1748
Time for Getting  4425  random elements:  0:00:00.007338
Batch Time:  0:00:00.140965
batch Number:  1749
Time for Getting  4425  random elements:  0:00:00.007346
Batch Time:  0:00:00.150913
batch Number:  1750
Time for Getting  4425  random elements:  0:00:00.007349
Batch Time:  0:00:00.142853
batch Number:  1751
Time for Getting  4425  random elements:  0:00:00.007577
Batch Time:  0:00:00.141207
batch Number:  1752
Time for Getting  4425  random elements:  0:00:00.007312
Batch Time:  0:00:00.140811
batch Number:  1753
Time for Getting  4425  random elements:  0:00:00.007295
Batch Time:  0:00:00.139629
batch Number:  1754
Time fo

Batch Time:  0:00:00.143723
batch Number:  1825
Time for Getting  4425  random elements:  0:00:00.007503
Batch Time:  0:00:00.145620
batch Number:  1826
Time for Getting  4425  random elements:  0:00:00.007676
Batch Time:  0:00:00.139689
batch Number:  1827
Time for Getting  4425  random elements:  0:00:00.007723
Batch Time:  0:00:00.138619
batch Number:  1828
Time for Getting  4425  random elements:  0:00:00.007825
Batch Time:  0:00:00.139559
batch Number:  1829
Time for Getting  4425  random elements:  0:00:00.007570
Batch Time:  0:00:00.138968
batch Number:  1830
Time for Getting  4425  random elements:  0:00:00.007671
Batch Time:  0:00:00.141980
batch Number:  1831
Time for Getting  4425  random elements:  0:00:00.007337
Batch Time:  0:00:00.136345
batch Number:  1832
Time for Getting  4425  random elements:  0:00:00.007218
Batch Time:  0:00:00.137700
batch Number:  1833
Time for Getting  4425  random elements:  0:00:00.007705
Batch Time:  0:00:00.144799
batch Number:  1834
Time fo

Batch Time:  0:00:00.142077
batch Number:  1905
Time for Getting  4425  random elements:  0:00:00.007348
Batch Time:  0:00:00.145841
batch Number:  1906
Time for Getting  4425  random elements:  0:00:00.007740
(array([[4.610000e+02, 6.490000e+02, 9.700000e+01, 7.460000e+02,
        3.003800e+04, 9.060000e+02, 0.000000e+00],
       [4.000000e+00, 6.288500e+04, 3.396000e+03, 6.626000e+03,
        1.645960e+05, 7.870000e+02, 0.000000e+00],
       [2.000000e+00, 2.586800e+04, 4.150000e+03, 8.662000e+03,
        1.431000e+04, 3.200000e+02, 0.000000e+00],
       [1.000000e+00, 2.504000e+03, 3.576000e+03, 5.111100e+04,
        2.472000e+03, 3.762000e+03, 0.000000e+00],
       [4.000000e+00, 4.148000e+04, 2.762000e+03, 8.237000e+03,
        7.941925e+06, 2.376000e+03, 0.000000e+00],
       [3.000000e+00, 5.700000e+02, 4.850000e+02, 9.655000e+03,
        1.532000e+03, 3.473700e+04, 0.000000e+00],
       [1.000000e+00, 1.100000e+01, 3.000000e+00, 6.640000e+02,
        1.280000e+02, 1.548000e+03,

KeyboardInterrupt: 